In [1]:
from torchvision import models
resnet = models.resnet50(pretrained=True)

print(resnet)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\YSH/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
import torch 

sample_x = torch.randn((1, 3, 255, 255))

out = resnet(sample_x)

out.shape

torch.Size([1, 1000])

In [5]:
import torch
import torch.nn as nn

class ResembleGAN(nn.Module):
    def __init__(self, device):
        super(ResembleGAN, self).__init__()

        self.resnet = models.resnet50(pretrained=True).to(device)

        self.stylegan = torch.load("./models/stylegan2-afhqwild-512x512.pkl").to(device)

        self.prepare_network()

    def prepare_network(self):
        for para in self.stylegan.parameters():
            para.requires_grad = False
        
    def forward(self, x):
        x = x.reshape(x.shape[0], 3, 255, 255)

        latent_vector = self.resnet(x)

        estimation = self.stylegan(latent_vector)

        return estimation, latent_vector